In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

RNNやLSTMを使った楽しいアプリケーション
- コーパスを使って学習した言語モデルを使った文章生成とその改良  
- seq2seq(sequence to sequence)
    - 2つのRNNを組み合わせて実装する
    - 機械翻訳やチャットボット，メールの自動返信などに使える

# 言語モデルを使った文章生成

## RNNによる文章生成の手順
例えば，
<div style="text-align: center">you say goodbye and I say Hello . </div>  

というコーパスで学習を行った言語モデルでは，「I」を入力すると次に出現する単語は「say」の確率が高くなる．  
このことを利用して文章生成ができる．  
<br>
もっとも確率の高い単語を選び続ければ結果は一意に定まってしまう．(決定的なアルゴリズム)  
しかし，確率的に単語を選べば毎回異なった文章が生成できる．  
この作業を望む回数だけ，または<eos\>のような文末記号が出現するまで繰り返す．  
このようにしてできた文章は，訓練データには存在しない文章の並びになりうる．  

## 文章生成の実装
前章で実装したRnnlmクラスをベースに，それを継承してRnnlmGenクラスを生成する.  
このクラスに文章生成を行うメソッドを追加する．

In [6]:
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        """
        言語モデルから文章の生成を行う．
        
        Parameters
        --------------
        start_id: int
            文章生成の際の最初の単語のID
        skip_ids: Array<int>
            文章生成の際サンプリングしたくない単語のID
            PTBデータセットの<unk>(レアな単語)やN(数値)などの前処理されたデータをサンプリングしないようにする用途で使う
        sample_size: int = 100
            生成する文章の長さ
        """
        word_ids = [start_id]
        
        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1) # 文章の長さ1,バッチサイズも1の入力データに変換
            score = self.predict(x) # 次の単語を予測
            p = softmax(score.flatten()) # 予測結果の確率分布を取得(softmaxで正規化)
            
            sampled = np.random.choice(len(p), size=1, p=p) # 予測結果の確率分布を使ってp個の単語の中から次の単語1個をサンプル
            
            if (skip_ids is None) or (sampled not in skip_ids): # skip_idが空ならSkip_IDの探索を行わない
                x = sampled # Skipワードでなければこの単語を次の探索に使う
                word_ids.append(int(x))
        
        return word_ids # 生成された文章

まずは重みパラメータがランダムな初期値の状態で文章作成を行わせてみる．

In [13]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('./ch06/Rnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you get the people.
 we 'll have offered sound to approach wood.
 the game 's effort has shown in australia or the additional year because of which the grocery circuit plans to assist in reins the vice securities known as directors of manufacturers hanover corp. 's employee plans to be provided impact on the problem.
 state the losses are bonuses at the number of expires tomorrow.
 the largest u.s. operations is increasingly offset by tuesday 's goal but expensive purchases by itself morgan competitive authorities said.
 roberti sciences studies typically trust to redeem steadily customers


初期値で文章を生成した結果の一例  
<br>
you heating destroying split yale proteins teacher flowing shoulder collection knowledgeable whitten pence cancer balance buildings geared profession growing tonight hud compounded franchisers still cooperative reitman settlement benjamin bail lenses johns sometimes processing quickly arab wins edge men asia louis rate statute short destruction founded contrast potato robust steadily net either shapiro contest begins participate buffer accounted cautiously travel embraced refunding merchandising passenger referring issuer shaking ring vintage largest john passion resign urged them poland sandinistas anderson native dominates tire bruce justified istat des dean westridge perceptions talk enough brisk handed airline foot panic sedan scores transport exodus 24-hour championship  
<br>
でたらめである．

学習済みのパラメータを使って文章を生成した結果の一例  
<br>
you get the people.
we 'll have offered sound to approach wood.
the game 's effort has shown in australia or the additional year because of which the grocery circuit plans to assist in reins the vice securities known as directors of manufacturers hanover corp. 's employee plans to be provided impact on the problem.
state the losses are bonuses at the number of expires tomorrow.
the largest u.s. operations is increasingly offset by tuesday 's goal but expensive purchases by itself morgan competitive authorities said.
roberti sciences studies typically trust to redeem steadily customers  
<br>
文法的に正しい文章もあるようだが，意味の通じない文章も散見される．

## さらに良い文章へ
